# Modelling Data

In [1]:
import pickle
import pandas as pd
from sklearn.compose import ColumnTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.linear_model import Lasso
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
import time
import numpy as np
from sklearn.metrics import make_scorer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import Ridge
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.ensemble import GradientBoostingRegressor

In [3]:
## loading in the cleaned df

df = pickle.load(open("C:/Users/harpr/Documents/GitHub/ds_salary_proj/pickle objects/jobs_df_cleaned.pkl", "rb"))

In [4]:
df

,salary,description,company_size,company_type,industry,revenue,company_rating,recommend_to_a_friend,ceo_approval,interview_difficulty,location_bin
0,144000,TikTok is the leading destination for short-fo...,1001 to 5000 employees,Private,Internet,NaN,4.1,79.0,100.0,2.7,CA
1,107000,THE ROLE\n\nTesla's mission is to accelerate t...,10000+ employees,Public,Transportation Equipment Manufacturing,$2 to $5 billion (CAD) per year,3.5,59.0,75.0,2.9,CA
2,150000,Facebook's mission is to give people the power...,10000+ employees,Public,Internet,$5 to $10 billion (CAD) per year,4.4,90.0,94.0,3.1,CA
3,130000,Job Description\n\nThe New York Times is commi...,1001 to 5000 employees,Public,News Outlets,$1 to $2 billion (CAD) per year,3.8,80.0,95.0,2.9,NY
4,66225,About Cerebri AI Cerebri AI CVX platform uses ...,1 to 50 employees,Private,Enterprise Software & Network Solutions,NaN,3.9,75.0,80.0,2.4,ON
...,...,...,...,...,...,...,...,...,...,...,...
200,115000,Summary\n\nImagine what you could do here. At ...,10000+ employees,Public,Computer Hardware & Software,$10+ billion (CAD) per year,4.2,83.0,93.0,3.0,TX
201,155000,Facebook's mission is to give people the power...,10000+ employees,Public,Internet,$5 to $10 billion (CAD) per year,4.4,90.0,94.0,3.1,WA
202,134000,Description\n\nExcited by using massive amount...,10000+ employees,Public,Internet,$10+ billion (CAD) per year,3.9,77.0,85.0,3.0,NJ
203,139000,PlayStation isn’t just the Best Place to Play ...,5001 to 10000 employees,Subsidiary or Business Segment,Video Games,$10+ billion (CAD) per year,3.7,75.0,73.0,2.7,CA


## Scoring Metric

To devise an appropriate scoring metric for model comparison, it was necessary to first consider  what good model performance would look like in the context of the model’s use case. The use case for a job salary predictor would be for job seekers to have an estimate of a position’s salary so they could negotiate a salary above said estimate. There is far less cost to a job seeker for negotiating a salary above what the position is worth than there is for negotiating a salary below what it’s worth. Therefore, a good model would be one that provides a prediction close to the actual salary, but favors overestimating salaries over underestimating them. To this end, the following scoring metric was used, where a lower score indicates better model performance:

$score$ = $(salary - salary_{estimated})^2$ +  $I(salary, salary_{estimated})$

$I(salary, salary_{estimated})$ = ${\begin{Bmatrix}0&salary_{estimated} > salary\\|salary - salary_{estimated}|&salary_{estimated} < salary\end{Bmatrix}}$

In [5]:
## defining the scoring metric to be used with model evaluation 

def scoring_metric_(y_true, y_predicted):
    
    ## Squared error component of the error
    
    error_1 = ((y_true - y_predicted)**2)
    
    ## additional penalty for underestimating the salary 
    
    error_2 = (y_true - y_predicted)
    error_2[error_2 <= 0] = 0
    
    ## adding and then averaging the errors 
    
    total_error = error_1 + error_2
    
    average_error = np.average(total_error)
    
    return average_error

In [6]:
## Creating a scikit learn scoring objet from the scoring function that will be passed as a parameter inside grid search instances. 

scoring_metric = make_scorer(scoring_metric_, greater_is_better = False)

## Models

Given the small size of the training set and the amount of missing values, it made sense to fit models to the data that were especially insusceptible to overfitting. This constraint precluded ordinary regression, KNN, decision tree and neural net models from use. Regularized regression, random forest and gradient boosted tree models remained as viable choices. The tree based models seemed particularly suitable for the task because of their ability to capture non-linear relationships in the data without adding on model parameters and increasing model variance.

The company size, revenue, company rating, recommend to a friend rating, ceo approval rating and interview difficulty rating features all had missing values. Given the small size of the training set, it was unfeasible to simply remove any rows with missing values because most contained atleast one. Thus, missing value imputation was needed. To start, a univariate imputation strategy (Scikit Learn SimpleImputer class) was used with each of the models. Missing values were imputed using the mean/median of a feature if the feature was continuous and with the most frequent value if the feature was categorical. Then, a multivariate imputation strategy (Scikit Learn IterativeImputer class) was tested with the best performing model from the univariate imputation trials. In multivariate imputation, missing values in a given feature were imputed by predicting them from a model trained on the remaining features.

While grid-searching for optimal model parameters based on cross-validation scores using the above defined scoring metric, parameter spaces to search were, in general, chosen so as to reduce model complexity and therefore variance, given the risk of overfitting. 

### Lasso with SimpleImputer

In [7]:
## Splitting the training set into the feature matrix (X) and the response matrix (Y)

X = df.iloc[:, 1:]
Y = df.iloc[:, 0]

In [ ]:
## Creating pipelines of transformers to be applied to various subsets of the feature matrix to preprocess the data.  

The job description column was converted into a bag of words representation to make it digestible for ML models.  A TF-IDF (term frequency - inverse document frequency) weighting was applied to the document-term matrix so that the presence or lack thereof of rarer terms would have a greater influence on the salary prediction.

In [8]:
 ## 'description' column will be transformed into a document-term matrix using the TfidfVectorizer transformer. 

text_columns = 'description'
text_pipe =Pipeline(steps = [("tfidf", TfidfVectorizer(token_pattern =  r"[a-zA-Z0-9\+\-\.]+"))])

Categorical features -- the company size, company type, industry, revenue and location -- were one hot encoded to make them digestible for ML models. The company size, type and revenue features all had a modest number of levels, so high cardinality was not an issue. The industry feature, however, had many levels and therefore posed a problem. There were a few common industries and then single instances of many different industries. Encountering unseen levels at the time of prediction was not so much of an issue because most of the major industries were present in the training set. Overfitting the model to the various levels was, however, an . Unfortunately, there was no way to group together the single instances of many different industries without significantly reducing the granularity of information contained in the column. Therefore, the one-hot encoded industry feature was left as is. 

In [ ]:
## columns of continuous variables will have missing values imputed using the SimpleImputer transformer

continuous_columns = ['company_rating', 'recommend_to_a_friend', 'ceo_approval', 'interview_difficulty']
continuous_pipe =Pipeline(steps = [("impute_continuous", SimpleImputer())])

## Columns of categorical variables will have missing values imputed and will then be one-hot encoded using the OneHotEncoder transformer. 

categorical_columns = ['company_size', 'company_type', 'industry', 'revenue', 'location_bin']
categorical_pipe =Pipeline(steps = [("impute_categorical", SimpleImputer(strategy = 'most_frequent')), ('one_hot', OneHotEncoder(handle_unknown = 'ignore'))])

In [9]:
## Combining the various pipelines inside a ColumnTransformer object which will apply the transformations (pipelines) to their respective columns in parallel and then concatenate the outputs together. 

lasso_preprocess= ColumnTransformer([("text", text_pipe, text_columns), ("num", continuous_pipe, continuous_columns), ("cat", categorical_pipe, categorical_columns)], remainder = 'passthrough')

## Combining the column transformer object with the lasso estimator inside another pipeline, so that the output of the column transformer -- the processed feature matrix -- is passed into the lasso estimator. 

lasso_pipe = Pipeline(steps = [("lasso_preprocess", lasso_preprocess), ("lasso", Lasso(normalize = True, warm_start=True))])

## Defining the paramter grid to be used inside the grid search object.

lasso_parameters = {'lasso_preprocess__text__tfidf__ngram_range' : [(1,2), (1,3)], 'lasso_preprocess__text__tfidf__min_df' : [0.0, 0.05, 0.10], 'lasso_preprocess__text__tfidf__max_df' : [0.80, 0.85, 0.90], 'lasso_preprocess__num__impute_continuous__strategy': ['mean', 'median'], 'lasso_preprocess__num__impute_continuous__add_indicator' : [True, False], 'lasso_preprocess__cat__impute_categorical__add_indicator' : [True, False], "lasso__alpha" : [1.0, 2.0, 4.0]}

## Instantiating the grid search object with the parameter grid,  pipeline of columntransformer + estimator and the custom scoring object. 

lasso_gs = GridSearchCV(lasso_pipe, lasso_parameters, scoring = scoring_metric, return_train_score = True, n_jobs = -1, verbose=10)    

In [ ]:
## Fitting the grid search object to the data. 

lasso_gs.fit(X,Y)

#### Score

In [11]:
## Retrieving the score (based on the custom scoring metric) achieved by the best hyperparameter combination.

lasso_gs.best_score_

-511853083.4323797

### Ridge with SimpleImputer

In [12]:
## Reusing the lasso preprocessing pipeline with the ridge estimator 

ridge_pipe = Pipeline(steps = [("ridge_preprocess", lasso_preprocess), ("ridge", Ridge(normalize = True))])
ridge_parameters = {'ridge_preprocess__text__tfidf__ngram_range' : [(1,2), (1,3)], 'ridge_preprocess__text__tfidf__min_df' : [0.0, 0.05, 0.10], 'ridge_preprocess__text__tfidf__max_df' : [0.80, 0.85, 0.90], 'ridge_preprocess__num__impute_continuous__strategy': ['mean', 'median'], 'ridge_preprocess__num__impute_continuous__add_indicator' : [True, False], 'ridge_preprocess__cat__impute_categorical__add_indicator' : [True, False], "ridge__alpha" : [1.0, 2.0, 4.0, 8.0]}
ridge_gs = GridSearchCV(ridge_pipe, ridge_parameters, scoring = make_scorer(scoring_metric_, greater_is_better = False), return_train_score = True, n_jobs = -1, verbose=10)    

In [ ]:
ridge_gs.fit(X, Y)

#### Score

In [150]:
ridge_gs.best_score_

-632986257.9196513

### Random Forest with SimpleImputer

In [13]:
## Reusing the lasso preprocessing pipeline with the random forest estimator 

rf_pipe = Pipeline(steps = [("rf_preprocess", lasso_preprocess), ("rf", RandomForestRegressor(n_estimators = 500, oob_score = True, n_jobs = -1))])

rf_parameters = {'rf_preprocess__text__tfidf__ngram_range' : [(1,2), (1,3)], 'rf_preprocess__text__tfidf__min_df' : [0.0, 0.05, 0.10], 'rf_preprocess__text__tfidf__max_df' : [0.80, 0.85, 0.90], 'rf_preprocess__num__impute_continuous__strategy': ['mean', 'median'], 'rf_preprocess__num__impute_continuous__add_indicator' : [True], 'rf_preprocess__cat__impute_categorical__add_indicator' : [True], 'rf__max_features': [175, 350, 700, 1400]}

rf_gs = GridSearchCV(rf_pipe, rf_parameters, scoring = scoring_metric, return_train_score = True, n_jobs = -1, verbose=10)    

In [ ]:
rf_gs.fit(X, Y)

#### Score

In [15]:
rf_gs.best_score_

-412399743.3754406

### Gradient Boosted Tree with SimpleImputer

In [16]:
## Reusing the lasso preprocessing pipeline with the gradient boosted tree estimator 

gb_pipe = Pipeline(steps = [("gb_preprocess", lasso_preprocess), ("gb", GradientBoostingRegressor())])
gb_parameters = {'gb_preprocess__text__tfidf__ngram_range' : [(1,2), (1,3)], 'gb_preprocess__text__tfidf__min_df' : [0.0, 0.05, 0.10], 'gb_preprocess__text__tfidf__max_df' : [0.80, 0.85, 0.90], 'gb_preprocess__num__impute_continuous__strategy': ['median'], 'gb_preprocess__num__impute_continuous__add_indicator' : [True], 'gb_preprocess__cat__impute_categorical__add_indicator' : [True], 'gb__learning_rate': [0.1, 0.05, 0.025], 'gb__n_estimators' : [100, 200, 400], 'gb__max_features' : [None, 'sqrt'], 'gb__max_depth' : [1,2,3]}
gb_gs = GridSearchCV(gb_pipe, gb_parameters, scoring = make_scorer(scoring_metric_, greater_is_better = False), return_train_score = True, n_jobs = -1, verbose=10)    

In [ ]:
gb_gs.fit(X, Y)

#### Score

In [159]:
gb_gs.best_score_

-390555067.43253165

### Gradient Boosted Tree with IterativeImputer

Using Scikit-learn's IterativeImputer class in conjunction with the OneHotEncoder class presents a problem. The IterativeImputer class requires all categorical variables to be one-hot encoded first, but the OneHotEncoder class requires missing values in categorical variables be imputed first. To work around this problem, we use the pandas get_dummies method, which can handle missing values, to encode the categorical variables. The get_dummies method creates a new bindary feature for each level of a categorical variable and one additional feature to indicate a missing value. For example, imagine a categorical variable has 3 levels. A row with a missing value for this variable would be encoded as [0, 0, 0, 1], where the first 3 columns correspond to the 3 levels of the variable and the last column corresponds to the missing value indicator feature. To make this encoding compatible with the IterativeImputer, we can replace all 0 values with np.nan and drop the missing value indicator feature entirely. This would make the row look like this: [NaN, NaN, NaN]. Now, the IterativeImputer can simply predict a value (ideally only 0 or 1) at each level of the categorical variable to indicate whether or not it thinks the missing value belongs to that level. For example, the final output after imputation could be [0, 0, 1], indicating that the IterativeImputer thinks that the missing value is the 3rd level of the categorical variable. 

In [17]:
## Creating a one hot encoded version of the feature matrix, with an additional level for each feature to indicate if a row is missing a value for that feature.  

X_one_hot = pd.get_dummies(X, dummy_na = True, columns = ['company_size', 'company_type', 'industry', 'revenue', 'location_bin'])
X_one_hot

,description,company_rating,recommend_to_a_friend,ceo_approval,interview_difficulty,company_size_1 to 50 employees,company_size_10000+ employees,company_size_1001 to 5000 employees,company_size_201 to 500 employees,company_size_5001 to 10000 employees,...,location_bin_NY,location_bin_ON,location_bin_OR,location_bin_Other,location_bin_QC,location_bin_Scotland,location_bin_TX,location_bin_VA,location_bin_WA,location_bin_nan
0,TikTok is the leading destination for short-fo...,4.1,79.0,100.0,2.7,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
1,THE ROLE\n\nTesla's mission is to accelerate t...,3.5,59.0,75.0,2.9,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Facebook's mission is to give people the power...,4.4,90.0,94.0,3.1,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Job Description\n\nThe New York Times is commi...,3.8,80.0,95.0,2.9,0,0,1,0,0,...,1,0,0,0,0,0,0,0,0,0
4,About Cerebri AI Cerebri AI CVX platform uses ...,3.9,75.0,80.0,2.4,1,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
200,Summary\n\nImagine what you could do here. At ...,4.2,83.0,93.0,3.0,0,1,0,0,0,...,0,0,0,0,0,0,1,0,0,0
201,Facebook's mission is to give people the power...,4.4,90.0,94.0,3.1,0,1,0,0,0,...,0,0,0,0,0,0,0,0,1,0
202,Description\n\nExcited by using massive amount...,3.9,77.0,85.0,3.0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
203,PlayStation isn’t just the Best Place to Play ...,3.7,75.0,73.0,2.7,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0


In [18]:
## Finding indices for rows with missing company size value.
## Filling in np.nan as the value at each level of the company size variable for each row with a missing company size. 

missing_company_size_indices = X_one_hot[(X_one_hot['company_size_nan'] == 1)].index

X_one_hot.iloc[missing_company_size_indices, 5:12] = np.nan

In [19]:
## Dropping the missing value indicator feature now that missingness has been encoded with np.nan at each feature level. 

X_one_hot = X_one_hot.drop(columns = 'company_size_nan')

In [20]:
## Finding indices for rows with revenue value.
## Filling in np.nan as the value at each level of the revenue variable for each row with a missing revenue. 


missing_revenue_indices = X_one_hot[(X_one_hot['revenue_nan'] == 1)].index

X_one_hot.iloc[missing_revenue_indices, 69:80] = np.nan

In [21]:
## Dropping the missing value indicator feature now that missingness has been encoded with np.nan at each feature level. 

X_one_hot = X_one_hot.drop(columns = 'revenue_nan')

In [22]:
## Dropping all other missing value indicator features for the other categorical variables. 

X_one_hot = X_one_hot.drop(columns = ['company_type_nan', 'industry_nan', 'location_bin_nan'])

In [25]:
## Missing values will be imputed using the IterativeImputer class.

ct = ColumnTransformer([("impute", IterativeImputer(estimator = RandomForestRegressor(n_estimators = 500, n_jobs = -1), add_indicator = True), slice(1, len(X_one_hot.columns))), ("text", TfidfVectorizer(token_pattern =  r"[a-zA-Z0-9\+\-\.]+", ngram_range = (1,3), min_df = 0.05, max_df = 0.8), 'description')], remainder = 'passthrough')

gb_ii_pipe = Pipeline(steps = [("ct", ct), ("gb", GradientBoostingRegressor(n_estimators = 200, max_features = None, max_depth = 1, learning_rate = 0.05))])

gb_ii_parameters = {'ct__impute__estimator__max_features' : [25, 50, 75, 100]}

gb_ii_gs = GridSearchCV(gb_ii_pipe, gb_ii_parameters, scoring = make_scorer(scoring_metric_, greater_is_better = False), return_train_score = True, n_jobs = -1, verbose=10)

In [ ]:
gb_ii_gs.fit(X_one_hot, Y)

#### Score

In [27]:
gb_ii_gs.best_score_

-432223502.61122465

### Saving Modelling Pipeline

In [167]:
## Pickling the best estimator (a pipeline object) from the grid search procedure for the gradient boosted tree model with SimpleImputer.

pickle.dump(gb_gs.best_estimator_, open("gb_pipeline.pkl", "wb"))